In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
!pip install datasets

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from datasets import Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MAKATÖÖ/korpus.csv', header=None, names=["label", "text"])

In [ ]:
df

,label,text
0,neutraalne,Tänasest juhib riigi tugiteenuste keskust Tarm...
1,neutraalne,Leppoja töötas viimased neli aastat EASi juhat...
2,neutraalne,Eelnevalt töötas ta erinevatel ametikohtadel E...
3,neutraalne,Riigi Tugiteenuste Keskus on rahandusministeer...
4,neutraalne,Keskus osutab hetkel finants- ning personali- ...
...,...,...
2117,negatiivne,Jõhkra käitumisega nooruk on varem kriminaalko...
2118,negatiivne,Ilutulestik vigastas Tallinnas kolme inimest
2119,negatiivne,Tallinna kiirabil tuli teisipäeva õhtul ja ööl...
2120,negatiivne,Tallinna kiirabi peaarsti Raul Adlase sõnul sa...


In [ ]:
sentiment_numbers = {'negatiivne': 0, 'neutraalne': 1, 'positiivne': 2}
df['label'] = df['label'].replace(sentiment_numbers)
df

,label,text
0,1,Tänasest juhib riigi tugiteenuste keskust Tarm...
1,1,Leppoja töötas viimased neli aastat EASi juhat...
2,1,Eelnevalt töötas ta erinevatel ametikohtadel E...
3,1,Riigi Tugiteenuste Keskus on rahandusministeer...
4,1,Keskus osutab hetkel finants- ning personali- ...
...,...,...
2117,0,Jõhkra käitumisega nooruk on varem kriminaalko...
2118,0,Ilutulestik vigastas Tallinnas kolme inimest
2119,0,Tallinna kiirabil tuli teisipäeva õhtul ja ööl...
2120,0,Tallinna kiirabi peaarsti Raul Adlase sõnul sa...


In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
small_train_dataset = dataset["train"]
small_test_dataset = dataset["test"]

In [ ]:
small_train_dataset[0]

{'label': 2,
 'text': 'Seaduses saab selgemaks see, et kallasrajal peab olema tagatud võimalus liikumiseks ka juhul, kui tegu on näiteks maaparanduskraavidega või näiteks rohumaa hooldamisega, kus loomade karjatamiseks on aed rajatud ja selliseid takistusi on võimalik rajada ainult juhul kui kohalik omavalitsus on selleks eelnevalt nõusoleku andnud. Nii peaks kallasrajal liikumine saama klaarimaks ja selgemaks, leidis Pentus-Rosimannus.'}

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/est-roberta")
model = AutoModelForSequenceClassification.from_pretrained("EMBEDDIA/est-roberta", num_labels=3)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/est-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#model.config.problem_type = "multi_label_classification"

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512) ## data go miss

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1697 [00:00<?, ? examples/s]

Map:   0%|          | 0/425 [00:00<?, ? examples/s]

In [ ]:
print("Batch size of tokenized_train dataset:", len(tokenized_train))
print("Batch size of tokenized_test dataset:", len(tokenized_test))

Batch size of tokenized_train dataset: 1697
Batch size of tokenized_test dataset: 425


In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]

    return {"accuracy": accuracy, "f1": f1}

In [ ]:
#!pip install transformers[torch]

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
#!pip uninstall transformers accelerate
!pip install transformers accelerate

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.469500
1000,0.080100


TrainOutput(global_step=1070, training_loss=0.2596386111785318, metrics={'train_runtime': 459.0471, 'train_samples_per_second': 36.968, 'train_steps_per_second': 2.331, 'total_flos': 1028501613711054.0, 'train_loss': 0.2596386111785318, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

<ipython-input-77-6f0f0b6fb5c5>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 1.173004388809204,
 'eval_accuracy': 0.7835294117647059,
 'eval_f1': 0.7795046244649099,
 'eval_runtime': 4.3322,
 'eval_samples_per_second': 98.102,
 'eval_steps_per_second': 6.232,
 'epoch': 10.0}

In [ ]:
model.save_pretrained('roBertaSentiment')